<a href="https://www.kaggle.com/code/ayfait/cmc-gainers-notebook-starting-07-10-24?scriptVersionId=187818824" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

I'm writing this first code on Kaggle to create a program that will compile a list of **Coinmarketcap daily gainers**.
I already wrote a code for the Top10 gainers treemap [here](https://github.com/AyFait/Cmc_Scrape_Top10_HeatMap_TreeMap)

In [ ]:
#installing chrome
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y

In [ ]:
#installing chromedriver
!wget https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


In [ ]:
#insatlling selenium
!pip install --upgrade jupyter-lsp
!pip install --upgrade selenium
!pip install webdriver-manager


In [ ]:
import pandas as pd
import time
from datetime import datetime
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode.
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")  # This line helps in some environments
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


# Test the WebDrive
url='https://coinmarketcap.com/gainers-losers/'
driver.get(url)
print(driver.title)


In [ ]:
#define url of page to exract data from
url='https://coinmarketcap.com/gainers-losers/'
driver.get(url)
time.sleep(5) #Sleep for few seconds so, by that time, the webpage gets loaded.
ranking =  driver.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div/div[2]/div/div[2]/div/div[1]/div/table')# get element by XPATH from element selection in inspect mode


In [ ]:
data = []  # Extract data
for index, rank in enumerate(ranking):
    rowData = rank.text.split('\n')
    #print(rowData) #This prints the extracted data all in a single row
    data.append(rowData)


In [ ]:
#Resahaping the data list to # cols
lst_cleaned = rowData[5:] #popping headers
expectedRows = len(lst_cleaned) // 4 #divide the list by no of expected cols to get no of expected rows
lst_array = np.array(lst_cleaned)#turning the simngle rowdata list into an array
reshpd = lst_array.reshape(int(expectedRows), 4) #to get row x col
reshpdtrimmed = reshpd[:, :-1]#'Price' '24h%' 'Vol(24h)' were merged together so I popped them
forth = np.array([row[3].split() for row in reshpd])#now splitting 'Price' '24h%' 'Vol(24h)' on their own
lst_n_forth = np.concatenate((reshpdtrimmed, forth), axis = 1)#joining them back together

In [ ]:
#Adding TimeStamp for each row
timing = []
for i in range(len(lst_n_forth)):
    timestamp = datetime.utcnow().strftime('%m/%d/%Y %H:%M:%S UTC')
    timing.append(timestamp)
#print(timing)
timing_array = np.array(timing)    
timingreshpd = timing_array.reshape(30, 1)
timed_array = np.concatenate((timingreshpd, lst_n_forth), axis = 1)#coming together making the perfect array with timestamp
#print(len(timed_array))
#print(timed_array)

In [ ]:
#passing to nympy Dataframe to get tabular form
df = pd.DataFrame(timed_array,index=np.arange(1, len(timed_array)+1), columns=['Time_Stamp', 'CMC_Rank', 'Name', 'Symbol', 'Price($)', '24h%', '24h_Vol($)'])
df['Price($)'] = df['Price($)'].str.lstrip('$').astype(float)
df['24h%'] = df['24h%'].str.rstrip('%').astype(float)
df['24h_Vol($)'] = df['24h_Vol($)'].str.lstrip('$').str.replace(',', '').astype(float)
#print(df)
timestamp = datetime.utcnow().strftime('%b %d, %Y %H:%M:%S UTC')
title = f"Top 10 CMC 24h% Increase for {timestamp}"
display_title = f"**{title}**"
# Display the title and DataFrame as a Markdown
from IPython.display import display, Markdown
display(Markdown(display_title))
display(df)


In [ ]:
#The first list created
df.to_csv(f'/kaggle/dataset/CMC_24h_Gainers_Starting_07_10_24 updated - {timestamp}.csv', index=False)

#To append to the list
#df.to_csv(f'/kaggle/dataset/CMC_24h_Gainers_Starting_07_10_24 updated - {timestamp}.csv',mode='a', header=False, index=False)

In [ ]:
#Setting up environment variables in order to use the kaggle CLI to automate upoloading of the dataset
import os
import json

from kaggle_secrets import UserSecretsClient


secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")

In [ ]:
#creating the folder that will hold the dataset and create the metadata file:
os.makedirs('/kaggle/dataset/', exist_ok=True)


# Change below
meta = dict(
    id="ayfait/my-dataset",
    title="CMC Gainers Dataset Starting_07_11_2024",
    isPrivate=False,
    licenses=[dict(name="Apache 2.0")]
)

with open('/kaggle/dataset/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

In [ ]:
#To upload the dataset created inside /kaggle/dataset for the first time:
!kaggle datasets create -p "/kaggle/dataset" --dir-mode zip

#To push a new version of the dataset 
#!kaggle datasets version -p "/kaggle/dataset" -m "Updated via notebook" --dir-mode zip